In [ ]:
#Step 1: Define the function for the UDF

def double(input_value: int):
  return input_value*2

In [ ]:
#Step 2: we need to import data type for returned value
from snowflake.snowpark.types import IntegerType


In [ ]:
#Step 3:  Register UDF in Snowflake and Upload UDF to Snowflake
#connection to Snowflake account
from snowflake.snowpark import Session
import os
import base64
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend

# Load private key from file with passphrase
with open(<<filepath>>, 'rb') as key_file:
    private_key = serialization.load_pem_private_key(
        key_file.read(),
        password=b"<<passphrase>>",
        backend=default_backend()
    )

# Convert private key to DER format and base64 encode it for Snowflake
private_key_der = private_key.private_bytes(
    encoding=serialization.Encoding.DER,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
 )
private_key_b64 = base64.b64encode(private_key_der).decode()

connection_parameters = {
    "account": "<<account>>",
    "user": "<<user>>",
    "private_key": private_key_b64,
    "role": "<<role>>",  # optional
    "warehouse": "<<warehouse>>",  # optional
    "database": "<<database>>",  # optional
    "schema": "PUBLIC",  # optional
} 

snowpark_session = Session.builder.configs(connection_parameters).create() 

snowpark_session.udf.register(
    func = double
  , return_type = IntegerType()
  , input_types = [IntegerType()]
  , is_permanent = True
  , name = 'SNOWPARK_DOUBLE'
  , replace = True
  , stage_location = '@UDF_STAGE'
)

In [ ]:
#Step 4: Test the UDF by calling 
snowpark_session.sql('SELECT SNOWPARK_DOUBLE(2)').show()